# Classify Flowers Using Measurements

This example is from: https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/

The test problem we will be using in this tutorial is iris classification.

Based on the sepal length, sepal width, petal length and petal width of 150 observations of iris flowers, we need to implement a classification model to predict the species, which is one of setosa, versicolor or virginica.

It is a standard dataset where the species is known for all instances. As such we can split the data into training and test datasets and use the results to evaluate our algorithm implementation. Good classification accuracy on this problem is above 90% correct, typically 96% or better.

Here is a iris flower image:
<img src="iris_petal_sepal.png" />

# How to implement k-Nearest Neighbors in Python

This tutorial is broken down into the following steps:

1. Handle Data: Open the dataset from CSV and split into test/train datasets.
2. Similarity: Calculate the distance between two data instances.
3. Neighbors: Locate k most similar data instances.
4. Response: Generate a response from a set of data instances.
5. Accuracy: Summarize the accuracy of predictions.
6. Main: Tie it all together.


## 1. Handle Data

The first thing we need to do is load our data file. The data is in CSV format without a header line or any quotes. We can open the file with the open function and read the data lines using the reader function in the csv module.

In [1]:
!pip3 install pandas
import pandas as pd
pd.__version__

url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
df = pd.read_csv(url, header=None)
print(df.head())

Defaulting to user installation because normal site-packages is not writeable
     0    1    2    3            4
0  5.1  3.5  1.4  0.2  Iris-setosa
1  4.9  3.0  1.4  0.2  Iris-setosa
2  4.7  3.2  1.3  0.2  Iris-setosa
3  4.6  3.1  1.5  0.2  Iris-setosa
4  5.0  3.6  1.4  0.2  Iris-setosa


Next we need to split the data into a training dataset that kNN can use to make predictions and a test dataset that we can use to evaluate the accuracy of the model.

We first need to convert the flower measures that were loaded as strings into numbers that we can work with. Next we need to split the data set randomly into train and datasets. A ratio of 67/33 for train/test is a standard ratio used.

Pulling it all together, we can define a function called loadDataset that loads a CSV with the provided filename and splits it randomly into train and test datasets using the provided split ratio.

In [2]:
import pandas as pd
import random


def loadDataset(filename, split):
	trainingSet=[]
	testSet=[]
	df = pd.read_csv(url, header=None)
	array = df.to_numpy()
	random.shuffle(array)
	training_len = int(len(array)*split)
	trainingSet = array[:training_len]
	testSet = array[training_len:]
	return trainingSet, testSet


We can test this function out with our iris dataset, as follows:

In [3]:
trainingSet=[]
testSet=[]
url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
trainingSet, testSet = loadDataset(url, 0.66)

# repr() Return a string containing a printable representation
print('Train: ' + repr(len(trainingSet)))
print('Test: ' + repr(len(testSet)))

Train: 99
Test: 51


## 2. Similarity

In order to make predictions we need to calculate the similarity between any two given data instances. This is needed so that we can locate the k most similar data instances in the training dataset for a given member of the test dataset and in turn make a prediction.

Given that all four flower measurements are numeric and have the same units, we can directly use the Euclidean distance measure. This is defined as the square root of the sum of the squared differences between the two arrays of numbers (read that again a few times and let it sink in).

Additionally, we want to control which fields to include in the distance calculation. Specifically, we only want to include the first 4 attributes. One approach is to limit the euclidean distance to a fixed length, ignoring the final dimension.

Putting all of this together we can define the euclideanDistance function as follows:

In [4]:
import math
def euclideanDistance(instance1, instance2, length):
	distance = 0
	for x in range(length):
		distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)

We can test this function with some sample data, as follows:

In [5]:
data1 = [2, 2, 2, 2, 'a']
data2 = [4, 4, 4, 4, 'b']
distance = euclideanDistance(data1, data2, 4)
print('Distance: ' + repr(distance))

Distance: 4.0


## 3. Neighbors

Now that we have a similarity measure, we can use it collect the k most similar instances for a given unseen instance.

This is a straight forward process of calculating the distance for all instances and selecting a subset with the smallest distance values.

Below is the getNeighbors function that returns k most similar neighbors from the training set for a given test instance (using the already defined euclideanDistance function)

In [6]:
import operator 
def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append((trainingSet[x], dist))
    # key=operator.itemgetter(1) => order distances list based on second column (1)
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

We can test out this function as follows:

In [7]:
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5, 'b']
k = 1
neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)

[[4, 4, 4, 'b']]


## 4. Response

Once we have located the most similar neighbors for a test instance, the next task is to devise a predicted response based on those neighbors.

We can do this by allowing each neighbor to vote for their class attribute, and take the majority vote as the prediction.

Below provides a function for getting the majority voted response from a number of neighbors. It assumes the class is the last attribute for each neighbor.

In [8]:
import operator
def getResponse(neighbors):
    # classVotes is a dict, similar to JSON
    # e.g., {'a': 1, 'b': 2}
	classVotes = {}
	for x in range(len(neighbors)):
        # based on the last element
		response = neighbors[x][-1]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
    # reverse=True means the most voted item will be our response
    # itemgetter(1): get the first item, i.e. item with most votes
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]

We can test out this function with some test neighbors, as follows:

In [9]:
neighbors = [[1,1,1,1,'a'], [2,2,2,2,'a'], [3,3,3,3,'b']]
response = getResponse(neighbors)
print(response)

a


This approach returns one response in the case of a draw, but you could handle such cases in a specific way, such as returning no response or selecting an unbiased random response.

## 5. Accuracy

We have all of the pieces of the kNN algorithm in place. An important remaining concern is how to evaluate the accuracy of predictions.

An easy way to evaluate the accuracy of the model is to calculate a ratio of the total correct predictions out of all predictions made, called the classification accuracy.

Below is the getAccuracy function that sums the total correct predictions and returns the accuracy as a percentage of correct classifications.

In [10]:
def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] is predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

We can test this function with a test dataset and predictions, as follows:

In [11]:
testSet = [[1,1,1,1,'a'], [2,2,2,2,'a'], [3,3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


## 6. Main

We now have all the elements of the algorithm and we can tie them together with a main function.

Below is the example of implementing the kNN algorithm from scratch in Python.

Group Activity: You need to finish the TODO section.

In [12]:
# Example of kNN implemented from Scratch in Python

import pandas as pd
import random
import math
import operator
import numpy as np

def loadDataset(filename, split):
	trainingSet=[]
	testSet=[]
	df = pd.read_csv(url, header=None)
	array = df.to_numpy()
	random.shuffle(array)
	training_len = int(len(array)*split)
	trainingSet = array[:training_len]
	testSet = array[training_len:]
	return trainingSet, testSet


def euclideanDistance(instance1, instance2, length):
	distance = 0
	for x in range(length):
		distance += pow((instance1[x] - instance2[x]), 2)
	return math.sqrt(distance)

def getNeighbors(trainingSet, testInstance, k):
	distances = []
	length = len(testInstance)-1
	for x in range(len(trainingSet)):
		dist = euclideanDistance(testInstance, trainingSet[x], length)
		distances.append((trainingSet[x], dist))
	distances.sort(key=operator.itemgetter(1))
	neighbors = []
	for x in range(k):
		neighbors.append(distances[x][0])
	return neighbors

def getResponse(neighbors):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x][-1]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]

def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0
	
def main():
	# prepare data
	trainingSet=[]
	testSet=[]
	split = 0.67
	url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
	trainingSet, testSet = loadDataset(url, 0.66)
	print('Train set: ' + repr(len(trainingSet)))
	print('Test set: ' + repr(len(testSet)))
	# generate predictions
	predictions=[]
	k_range = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
	for k in k_range: 
		print(k)
	# loop through testSet
		for x in range(len(testSet)):
		# TODO starts here
		# get neighor between current test record and all training datasets
			neighbors = getNeighbors(trainingSet, testSet[x], k)
		#get response
			result = getResponse(neighbors)
		# append current prediction result to predictions list
			predictions.append(result)
		print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
		# TODO ends here
		accuracy = getAccuracy(testSet, predictions)
		print('Accuracy is ' + repr(accuracy) + '%')
	print(k)
main()







Train set: 99
Test set: 51
1
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
2
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
3
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
4
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
5
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
6
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
7
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
8
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
9
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
10
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
11
> predicted='Iris-versicolor', actual='Iris-virginica'
Accuracy is 76.47058823529412%
12


In [13]:
#HW1

def main():
	k_range = list(range(1,20))
	for k in k_range:        
		for i in range(5):
		# prepare data
			trainingSet=[]
			testSet=[]
			split = 0.67
			url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
			trainingSet, testSet = loadDataset(url, 0.66)
			print('Train set: ' + repr(len(trainingSet)))
			print('Test set: ' + repr(len(testSet)))
		# generate predictions
			
			knn = KNeighborsClassifier(n_neighbors=k)
			
			
			accuracy = getAccuracy(testSet, predictions)
			print('Accuracy is ' + repr(accuracy) + '%')

main()





Train set: 99
Test set: 51


NameError: name 'KNeighborsClassifier' is not defined

In [ ]:
##Testing with Sckit
!pip3 install -U scikit-learn

!pip3 install matplotlib
import pandas as pd
import random
import math
import operator
import numpy as np
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
df = pd.read_csv(url, header= 0, names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm','Species'])
print(df.head())
df.info()

X = df.drop('Species', axis=1)
y = df['Species']

k_range = list(range(1,20))
for k in k_range:        
scores =[]
accuracy = []
for k in k_range: 
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X,y)
    y_pred = knn.predict(X)
    scores.append(metrics.accuracy_score(y,y_pred))
plt.plot(k_range,scores)


In [ ]:
##Sckitit Testing and training data

import pandas as pd
import random
import math
import operator
import numpy as np
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

##instance 1

url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
df = pd.read_csv(url, header= 0, names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm','Species'])
#print(df.head())
#df.info()

X1 = df.drop('Species', axis=1)
y1 = df['Species']

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.66, random_state= 0)


k_range = list(range(1,21))
scores1 =[]
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train1,y_train1)
    y_pred1 = knn.predict(X_test1)
    scores1.append(metrics.accuracy_score(y_test1,y_pred1))
plt.plot(k_range,scores1)


##instance 2
url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
df = pd.read_csv(url, header= 0, names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm','Species'])
X2 = df.drop('Species', axis=1)
y2 = df['Species']

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.66, random_state=2)


k_range = list(range(1,21))
scores2 =[]
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train2,y_train2)
    y_pred2 = knn.predict(X_test2)
    scores2.append(metrics.accuracy_score(y_test2,y_pred2))
plt.plot(k_range,scores2)

##instance 3
url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
df = pd.read_csv(url, header= 0, names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm','Species'])
X3 = df.drop('Species', axis=1)
y3 = df['Species']

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.66, random_state= 2)


k_range = list(range(1,21))
scores3 =[]
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train3,y_train3)
    y_pred3 = knn.predict(X_test3)
    scores3.append(metrics.accuracy_score(y_test3,y_pred3))
plt.plot(k_range,scores3)

##instance 4
url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
df = pd.read_csv(url, header= 0, names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm','Species'])
X4 = df.drop('Species', axis=1)
y4 = df['Species']

X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size=0.66, random_state= 3)


k_range = list(range(1,21))
scores4 =[]
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train4,y_train4)
    y_pred4 = knn.predict(X_test4)
    scores4.append(metrics.accuracy_score(y_test4,y_pred4))
plt.plot(k_range,scores4)

##instance 5
url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
df = pd.read_csv(url, header= 0, names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm','Species'])
X5 = df.drop('Species', axis=1)
y5 = df['Species']

X_train5, X_test5, y_train5, y_test5 = train_test_split(X5, y5, test_size=0.66, random_state= 4)


k_range = list(range(1,21))
scores5 =[]
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train5,y_train5)
    y_pred5 = knn.predict(X_test5)
    scores5.append(metrics.accuracy_score(y_test5,y_pred5))
plt.plot(k_range,scores5)


In [ ]:
## HW 1 to be graded
url = 'https://raw.githubusercontent.com/ruiwu1990/CSCI_4120/master/KNN/iris.data'
df = pd.read_csv(url, header= 0, names = ['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm','Species'])
print(df.head())
df.info()

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.66, random_state= 5)
#X = df.drop('Species', axis=1)
#y = df['Species']

k_range = list(range(1,21))
#scores =[]
for k in k_range:
    for i in range(5):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.66)
        X = df.drop('Species', axis=1)
        y = df['Species']
        scores=[]
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train,y_train)
        y_pred = knn.predict(X_test)
        scores.append(metrics.accuracy_score(y_test,y_pred))
        #means= np.mean(str(k),str(i),scores)

        print(str(k),scores)
        
#Average Accuracy for K Vals
import csv
from io import StringIO

mean_acc = pd.read_csv('/home/naomi/Desktop/CSCI_4120/KNN/CSCI_4120_HW1/average_accuracy_for_k_values.csv')
mean_acc_df= pd.DataFrame(mean_acc, columns=['k_value','Average_Accuracy_Score'])
print(mean_acc)



